In [ ]:
# Función para clasificar una mano tipo póker
ClasificarMano <- function(mano) {
  # Contamos la frecuencia de cada dígito
  frecuencias <- table(mano)
  cantidades <- sort(as.numeric(frecuencias), decreasing = TRUE)

  if (length(cantidades) == 1) {
    return("Quíntuple")  # Todos los dígitos son iguales
  } else if (length(cantidades) == 2 && cantidades[1] == 4) {
    return("Cuádruple")
  } else if (length(cantidades) == 2 && cantidades[1] == 3) {
    return("Full")
  } else if (length(cantidades) == 3 && cantidades[1] == 3) {
    return("Tercia")
  } else if (length(cantidades) == 3 && cantidades[1] == 2 && cantidades[2] == 2) {
    return("Dos pares")
  } else if (length(cantidades) == 4) {
    return("Par")
  } else {
    return("Todos diferentes")
  }
}

In [ ]:
# ---------------------------------------
# Función principal para realizar la prueba de réplica póker
PruebaReplicaPoker <- function(serie) {
  # Paso 1: Preparamos los dígitos como caracteres y los agrupamos en bloques de 5
  serieTexto <- gsub("\\.", "", as.character(serie))  # Quitamos el punto decimal
  digitos <- unlist(strsplit(serieTexto, ""))  # Convertimos a lista de dígitos
  digitos <- digitos[digitos %in% as.character(0:9)]  # Solo dígitos válidos

  bloques <- split(digitos, ceiling(seq_along(digitos) / 5))  # Agrupamos en bloques de 5

  # Paso 2: Clasificamos cada bloque como una "mano" tipo póker
  clasificaciones <- sapply(bloques, function(bloque) {
    if (length(bloque) == 5) {
      ClasificarMano(bloque)
    } else {
      return(NULL)
    }
  })

  clasificaciones <- na.omit(clasificaciones)  # Quitamos bloques incompletos

  # Paso 3: Contamos las frecuencias observadas
  categorias <- c("Todos diferentes", "Par", "Dos pares", "Tercia", "Full", "Cuádruple", "Quíntuple")
  frecuenciasObservadas <- table(factor(clasificaciones, levels = categorias))

  # Paso 4: Frecuencias esperadas (según teoría del póker)
  n <- sum(frecuenciasObservadas)
  probabilidades <- c(0.3024, 0.5040, 0.1080, 0.0720, 0.0090, 0.0045, 0.0001)
  frecuenciasEsperadas <- probabilidades * n

  # Paso 5: Calculamos el estadístico Chi Cuadrado
  chiCuadrado <- sum((frecuenciasObservadas - frecuenciasEsperadas)^2 / frecuenciasEsperadas)
  gradosLibertad <- length(categorias) - 1
  valorCritico <- qchisq(0.95, df = gradosLibertad)
  rechazarHipotesis <- chiCuadrado > valorCritico

  # Paso 6: Resultado
  resultado <- list(
    clasificaciones = clasificaciones,
    frecuenciasObservadas = frecuenciasObservadas,
    frecuenciasEsperadas = frecuenciasEsperadas,
    chiCuadrado = chiCuadrado,
    valorCritico = valorCritico,
    gradosLibertad = gradosLibertad,
    rechazarHipotesis = rechazarHipotesis
  )

  return(resultado)
}

In [ ]:
# ---------------------------------------
# Interfaz de consola para el usuario

CatBienvenidaPoker <- function() {
  cat("\n==============================\n")
  cat("   Prueba de Réplica Poker\n")
  cat("==============================\n")
  cat("Introduce una serie de números [0,1) separados por comas (ej. 0.12345, 0.67890):\n")
}

In [ ]:
LeerSeriePoker <- function() {
  entrada <- readline(prompt = "Ingresa la serie: ")
  serieNumeros <- as.numeric(strsplit(entrada, ",")[[1]])

  if (any(is.na(serieNumeros)) || any(serieNumeros < 0 | serieNumeros >= 1)) {
    cat("\nError: Todos los valores deben estar en el intervalo [0, 1).\n")
    return(NULL)
  }

  return(serieNumeros)
}

In [ ]:
MostrarResultadoPoker <- function(resultado) {
  cat("\n==============================\n")
  cat("Resultados de la Prueba de Réplica Poker:\n")
  cat("==============================\n")
  cat("Frecuencias observadas por categoría:\n")
  print(resultado$frecuenciasObservadas)

  cat("\nFrecuencias esperadas por categoría:\n")
  print(round(resultado$frecuenciasEsperadas, 2))

  cat("\nEstadístico Chi Cuadrado calculado:\n")
  print(resultado$chiCuadrado)

  cat("\nGrados de libertad:\n")
  print(resultado$gradosLibertad)

  cat("\nValor crítico (al 95% de confianza):\n")
  print(resultado$valorCritico)

  cat("\n¿Se rechaza la hipótesis de aleatoriedad?\n")
  if (resultado$rechazarHipotesis) {
    cat("Sí, se rechaza la hipótesis (los dígitos no son aleatorios).\n")
  } else {
    cat("No, no se rechaza la hipótesis (los dígitos son aleatorios).\n")
  }
}

In [ ]:
# ---------------------------------------
# Función principal
PruebaReplicaPokerInteractiva <- function() {
  CatBienvenidaPoker()
  serie <- LeerSeriePoker()

  if (!is.null(serie)) {
    resultado <- PruebaReplicaPoker(serie)
    MostrarResultadoPoker(resultado)
  }
}

In [ ]:
# Ejecutar el programa
PruebaReplicaPokerInteractiva()